In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import pandas as pd
device = torch.device("cuda")

from train_autoencoders import*
from coupled_learning_mlp import*
from GAN import*
from Mlp import*
from Testing_function import*
from Autoencoders import*
from train_mapping_functions import*
from coupled_learning_new import*


In [ ]:
df=pd.read_csv('/home/alexgi/Workspace/REC_SYSTEMS/My_codes/Data/Rs_no3.csv')
Movies_source=df.values       # movies x users
del df
df=pd.read_csv('/home/alexgi/Workspace/REC_SYSTEMS/My_codes/Data/Rt_no3.csv')
Movies_target=df.values   

Movies_source=Movies_source/5
Movies_target=Movies_target/5
print('source domain sparsity:', np.count_nonzero(Movies_source==0)/(Movies_source.shape[0]*Movies_source.shape[1])*100)
print('target domain sparsity:', np.count_nonzero(Movies_target==0)/(Movies_target.shape[0]*Movies_target.shape[1])*100)

Movies_source = Movies_source[:,1:Movies_source.shape[1]]
Movies_target = Movies_target[:,1:Movies_target.shape[1]]

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MovieDataset(Dataset):
    def __init__(self, movies):
        self.movies = movies

    def __len__(self):
        return len(self.movies)

    def __getitem__(self, index):
        return index, self.movies[index, :]

class UserDataset(Dataset):
    def __init__(self, users):
        self.users = users

    def __len__(self):
        return self.users.shape[0]

    def __getitem__(self, index):
        return index, self.users[index, :]

In [ ]:
#Load Training and Testing data

Movies_source_train, Movies_source_test, Movies_target_train, Movies_target_test = train_test_split(Movies_source, Movies_target, test_size=0.2)
Users_source_train = Movies_source_train.T
Users_source_test  = Movies_source_test.T

Users_target_train = Movies_target_train.T
Users_target_test  = Movies_target_test.T

Users_source=Movies_source.T
# del df,Movies_s,Movies_t


In [ ]:

num_epochs = 120
learning_rate = 1e-3
l2_w = 1e-8

batch_size_m = 64
batch_size_u = 64
batch_size = batch_size_m

layer_1_dim = 64
layer_2_dim = 32
layer_3_dim = 16

##Source Domain ----------------------------------------------------------------------------------------------------------------------------------

dataloader_movies = DataLoader(MovieDataset(Movies_source_train), batch_size=batch_size_m, shuffle=False)
dataloader_users = DataLoader(UserDataset(Users_source_train), batch_size=batch_size_m, shuffle=False)


model_encoder_m_s = encoder(Users_source_train.shape[0],layer_1_dim, layer_2_dim, layer_3_dim).cuda()
model_decoder_m_s = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Users_source_train.shape[0]).cuda()

model_encoder_u_s = encoder(Movies_source_train.shape[0], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
model_decoder_u_s = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Movies_source_train.shape[0]).cuda()

train_latentfactors_autoencoders_new(model_encoder_m_s, model_decoder_m_s, model_encoder_u_s, model_decoder_u_s, num_epochs,
                                learning_rate, dataloader_movies, dataloader_users, torch.tensor(Movies_source_train), device, l2_w, 
                                weight_loss_movies = 0.5, weight_loss_users = 0.5, weight_loss_latent = 0.5)

del dataloader_movies, dataloader_users

##Traget DOmain ---------------------------------------------------------------------------------------------------------------------------------

dataloader_movies = DataLoader(MovieDataset(Movies_target_train), batch_size=batch_size_m, shuffle=False)
dataloader_users = DataLoader(UserDataset(Users_target_train), batch_size=batch_size_m, shuffle=False)

model_encoder_m_t = encoder(Users_target_train.shape[0], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
model_decoder_m_t = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Users_target_train.shape[0]).cuda()

model_encoder_u_t = encoder(Movies_target_train.shape[0], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
model_decoder_u_t = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Movies_target_train.shape[0]).cuda()

train_latentfactors_autoencoders_new(model_encoder_m_t, model_decoder_m_t, model_encoder_u_t, model_decoder_u_t, num_epochs,
                                learning_rate, dataloader_movies, dataloader_users, torch.tensor(Movies_target_train), device, l2_w, 
                                weight_loss_movies = 0.5, weight_loss_users = 0.5, weight_loss_latent = 0.5)

In [ ]:
#Dataloader for the mapping MLP ----------------------------------------------------------------------------------------------------
dataset = TensorDataset(torch.tensor(Movies_source_train), torch.tensor(Movies_target_train))
dataloader_Coupled = DataLoader(dataset, 64, shuffle=True)
num_epochs = 50
learning_rate = 1e-3
l2_w = 1e-8

generator = Generator(input_size=layer_3_dim, hidden_size=layer_3_dim*2, output_size=layer_3_dim).to(device)
discriminator = Discriminator(input_size=layer_3_dim, hidden_size=layer_3_dim*2).to(device)
train_GAN(generator, discriminator, model_encoder_m_s, model_encoder_m_t, dataloader_Coupled, device, num_epochs, learning_rate)


# generator = mlp_network(layer_3_dim, 32).cuda()
# train_MLP(generator,  model_encoder_m_s, model_encoder_m_t, num_epochs, learning_rate, dataloader_Coupled, device, l2_w)



In [ ]:
dataloader_movies_source = DataLoader(MovieDataset(Movies_source_train), batch_size=64, shuffle=True)
dataloader_users_target = DataLoader(UserDataset(Users_target_train), batch_size=64, shuffle=True)


num_epochs = 100
learning_rate = 1e-4
l2_w = 1e-9
loss_rmse, loss_mae, prec, recall = train_Coupled_model_Matrix_factorization_new_version(generator, model_encoder_m_s, model_encoder_u_t, num_epochs, learning_rate,
                                            dataloader_movies_source, dataloader_users_target, device, Movies_source_test, Movies_target_test,
                                            l2_w, Users_target_train, Movies_target_train)

print((loss_rmse), (loss_mae), prec, recall)

In [ ]:
import pandas as pd

# Path to your CSV file
csv_path = "/home/alexgi/Workspace/REC_SYSTEMS/My_codes/Data/Rs_no1.csv"

rating_matrix = pd.read_csv(csv_path, index_col=0)

# # Prepare to collect non-zero entries
# data_list = []

# # Iterate over each item in the DataFrame
# for item_id, row in rating_matrix.iterrows():
#     for user_id, rating in row.iteritems():
#         if rating != 0:  # Assuming zero is the value that signifies 'no rating'
#             data_list.append({'userId': user_id, 'itemId': item_id, 'rating': rating})

# # Convert list to DataFrame
# filtered_data = pd.DataFrame(data_list)
